In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as  sns

In [2]:
! pip install catboost

^C


   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
    --------------------------------------- 2.4/102.5 MB 12.2 MB/s eta 0:00:09
   - -------------------------------------- 3.1/102.5 MB 10.3 MB/s eta 0:00:10
   - -------------------------------------- 3.1/102.5 MB 10.3 MB/s eta 0:00:10
   - -------------------------------------- 3.4/102.5 MB 4.0 MB/s eta 0:00:25
   -- ------------------------------------- 5.8/102.5 MB 5.5 MB/s eta 0:00:18
   --- ------------------------------------ 8.1/102.5 MB 6.5 MB/s eta 0:00:15
   --- ------------------------------------ 8.4/102.5 MB 6.6 MB/s eta 0:00:15
   --- ------------------------------------ 8.4/102.5 MB 6.6 MB/s eta 0:00:15
   --- ------------------------------------ 8.4/102.5 MB 6.6 MB/s eta 0:00:15
   --- ------------------------------------ 8.9/102.5 MB 4.2 MB/s eta 0:00:23
   --- ------------------------------------ 10.2/102.5 MB 4.4 MB/s eta 0:00:21
   ---- ----------------------------------- 10.5/102.5 MB 4.2 MB/s 

In [14]:
from sklearn.metrics import r2_score,accuracy_score,mean_squared_error,mean_absolute_error,root_mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [5]:
df=pd.read_csv('data/stud.csv')

In [6]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [7]:
X=df.drop(columns='math_score')

In [8]:
y=df['math_score']

In [11]:
num_features=X.select_dtypes(exclude="object").columns
cat_features=X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()
ct=ColumnTransformer(
    [
        ("OHE",oh_transformer,cat_features),
        ("Sscaler",numeric_transformer,num_features)
    ]
)

In [12]:
X=ct.fit_transform(X)

In [13]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [22]:
def evalute_model(true,predict):
    mae=mean_absolute_error(true,predict)
    mse=mean_squared_error(true,predict)
    rmse=root_mean_squared_error(true,predict)
    r2_scores=r2_score(true,predict)
    return mae,mse,rmse,r2_scores

In [16]:
models={
    "linearregression":LinearRegression(),
    "ridge":Ridge(),
    "Lasso":Lasso(),
    "KNeighborsRegressor":KNeighborsRegressor(),
    "DecisonTree":DecisionTreeRegressor(),
    "RandomForestRegressor":RandomForestRegressor(),
    "AdaBoostRegressor":AdaBoostRegressor(),
    "SVR":SVR(),
    "XGBRegressor":XGBRegressor()
}

In [24]:
model_list=[]
r2_score_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_predict=model.predict(X_train)
    y_test_predict=model.predict(X_test)
    mae_train,mse_train,rmse_train,r2_score_train=evalute_model(y_train,y_train_predict)
    mae_test,mse_test,rmse_test,r2_score_test=evalute_model(y_test,y_test_predict)
    model_list.append(list(models.keys())[i])
    print("Model Performance for Training set")
    print("rmse {:.4f}".format(rmse_train))
    print("mae {:.4f}".format(rmse_train))
    print("mse {:.4f}".format(mse_train))
    print("r2_score {:.4f}".format(r2_score_train))

    print('---------------------------------------------------')

    print("Model Performance in test data")
    print("rmse {:.4f}".format(rmse_test))
    print("mae {:.4f}".format(mae_test))
    print("mse {:.4f}".format(mse_test))
    print("r2_score {:.4f}".format(r2_score_test))
    r2_score_list.append(r2_score_test)
    print("=="*15)


Model Performance for Training set
rmse 5.3243
mae 5.3243
mse 28.3485
r2_score 0.8743
---------------------------------------------------
Model Performance in test data
rmse 5.3960
mae 4.2158
mse 29.1167
r2_score 0.8803
Model Performance for Training set
rmse 5.3233
mae 5.3233
mse 28.3378
r2_score 0.8743
---------------------------------------------------
Model Performance in test data
rmse 5.3904
mae 4.2111
mse 29.0563
r2_score 0.8806
Model Performance for Training set
rmse 6.5938
mae 6.5938
mse 43.4784
r2_score 0.8071
---------------------------------------------------
Model Performance in test data
rmse 6.5197
mae 5.1579
mse 42.5064
r2_score 0.8253
Model Performance for Training set
rmse 5.7077
mae 5.7077
mse 32.5776
r2_score 0.8555
---------------------------------------------------
Model Performance in test data
rmse 7.2530
mae 5.6210
mse 52.6066
r2_score 0.7838
Model Performance for Training set
rmse 0.2795
mae 0.2795
mse 0.0781
r2_score 0.9997
-----------------------------------

In [25]:
pd.DataFrame(list(zip(model_list,r2_score_list)),columns=['Model_name','r2_score'])

,Model_name,r2_score
0,linearregression,0.880345
1,ridge,0.880593
2,Lasso,0.825320
3,KNeighborsRegressor,0.783813
4,DecisonTree,0.717020
5,RandomForestRegressor,0.852560
6,AdaBoostRegressor,0.850851
7,SVR,0.728600
8,XGBRegressor,0.827797
